## Introduction
This notebook performs exploratory data analysis on SNCF train delay data. We'll clean the data, explore patterns, and prepare it for modeling.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from fuzzywuzzy import process
import re

# Set visualization style - updated for compatibility with newer versions
plt.style.use("seaborn-v0_8-whitegrid")  # Updated style name
sns.set_theme()  # Use the default seaborn theme

## Data Loading and Initial Exploration
Let's load the dataset and take a first look at its structure.

In [ ]:
print("Loading dataset...")
df = pd.read_csv("dataset.csv", sep=";")

# Display basic information
print("\nDataset Overview:")
print(f"Shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())

# Check column info
print("\nColumn information:")
df.info()

# Check for missing values
print("\nMissing values:")
missing_values = df.isna().sum()
missing_pct = (df.isna().sum() / len(df)) * 100
missing_df = pd.DataFrame({"Missing Values": missing_values, "Percentage": missing_pct})
print(missing_df[missing_df["Missing Values"] > 0])

# Basic statistics
print("\nBasic statistics:")
print(df.describe())

## Data Cleaning and Preprocessing
Now we'll clean the data by handling missing values, converting data types, and creating additional features.

## Normalizing String Fields
To ensure consistency and improve data quality, we normalize the names of various string fields such as "Service," "Departure station," and "Arrival station." This process involves:

1. **Trimming Whitespace**: Removing any leading or trailing spaces.
2. **Standardizing Case**: Converting text to title case for uniformity.
3. **Removing Extra Spaces**: Replacing multiple spaces with a single space.

This normalization step helps in avoiding mismatches caused by variations in formatting and ensures that the data is clean and ready for further analysis.

In [ ]:
# === Normalize text fields before matching ===
def normalize_station_name(text):
    if isinstance(text, str):
        return " ".join(text.strip().title().split())
    return text


df["Service"] = df["Service"].apply(normalize_station_name)
df["Departure station"] = df["Departure station"].apply(normalize_station_name)
df["Arrival station"] = df["Arrival station"].apply(normalize_station_name)

## Building Reference Lists for Fuzzy Matching
In this step, we create reference lists of the most common station names and services. These lists will be used later for fuzzy matching to correct rare or inconsistent entries in the dataset. This ensures data consistency and improves the quality of our analysis.

In [ ]:
# === Build reference list using top most common station names in your dataset ===
top_departures = df["Departure station"].value_counts().head(59).index.tolist()
top_arrivals = df["Arrival station"].value_counts().head(59).index.tolist()
top_services = df["Service"].value_counts().head(2).index.tolist()

print("=== TOP DEPARTURES ===")
print(top_departures)

print("=== TOP ARRIVALS ===")
print(top_arrivals)

print("=== TOP SERVICES ===")
print(top_services)

## Fuzzy Matching and Applying Fixes
To ensure data consistency, we use fuzzy matching to correct rare or inconsistent entries in the dataset. This involves:

1. **Fuzzy Matching**: Comparing each entry with a reference list of common values and finding the closest match based on a similarity score.
2. **Threshold-Based Correction**: If the similarity score exceeds a predefined threshold, the entry is corrected to the closest match; otherwise, it remains unchanged.
3. **Application**: This process is applied to the "Service," "Departure station," and "Arrival station" columns using the reference lists created earlier.

This step helps in standardizing the data and improving its quality for further analysis.

In [ ]:
# === Fuzzy matching function to auto-correct rare entries ===

def correct_station_from_top(name, top_list, threshold=75):
    if pd.isna(name):
        return name
    match, score = process.extractOne(name, top_list)
    if score < threshold:
        print(
            f"Correcting '{name}' to '{match}' - Score: {score} (Threshold: {threshold})"
        )
    # If the score is above the threshold, return the match; otherwise, return the original name
    return match if score >= threshold else name


# === Apply corrections using internal clean station list ===
df["Service"] = df["Service"].apply(lambda x: correct_station_from_top(x, top_services))
df["Departure station"] = df["Departure station"].apply(
    lambda x: correct_station_from_top(x, top_departures)
)
df["Arrival station"] = df["Arrival station"].apply(
    lambda x: correct_station_from_top(x, top_arrivals)
)

## Cleaning the Date Column
The `Date` column contains some invalid formats that need to be corrected. To clean this column, we:

1. **Fix Invalid Formats**: Replace invalid date formats (e.g., `YYYYMM` or `YYYY.MM`) with the correct `YYYY-MM` format.

In [ ]:
# First, clean the Date column by fixing invalid formats
df["Date"] = (
    # replace invalid formats with NaT (valid formats are YYYY-MM)
    df["Date"]
    .astype(str)
    .apply(
        lambda x: re.sub(r"(\d{4})([^-])(\d{2})", r"\1-\3", x)
        if re.match(r"\d{4}[^-]\d{2}", x)
        else x
    )
)

# Remove date in the future
df = df[
    df["Date"].apply(lambda x: pd.to_datetime(x, errors="coerce") <= datetime.now())
]

2. **Convert to Datetime**: Use robust error handling to convert the cleaned strings into proper datetime objects.
3. **Handle Missing Values**: Replace invalid or missing dates with `NaT` (Not a Time).

In [ ]:
# Convert Date column to datetime with robust error handling
print("Converting dates...")


# Use a custom conversion approach with error handling
def safe_date_convert(date_str):
    if pd.isna(date_str) or date_str == "" or date_str == "N/A":
        return pd.NaT  # Return Not a Time for empty/invalid entries

    try:
        return pd.to_datetime(date_str, format="%Y-%m")
    except ValueError:
        try:
            # Fallback to flexible parsing
            return pd.to_datetime(date_str)
        except ValueError:
            print(f"Could not parse date: {date_str}")
            return pd.NaT


# Apply the conversion
df["Date"] = df["Date"].apply(safe_date_convert)

4. **Validate date column**: we check the number of valid dates in the dataset and calculate the percentage of valid entries. Rows with invalid or missing dates are filtered out to create a clean dataset for further analysis.

This ensures that the `Date` column is consistent and ready for use in time-based analyses.


In [ ]:
# Check how many valid dates we have
valid_dates = df["Date"].notna().sum()
total_rows = len(df)
print(
    f"Successfully converted {valid_dates} out of {total_rows} dates ({valid_dates / total_rows:.2%})"
)

# Filter to keep only rows with valid dates if needed
df_clean = df[df["Date"].notna()].copy()

## Creating Additional Date Features
To enhance our analysis, we extract additional date-related features from the `Date` column. These features include:

1. **Year**: Extracted as a numeric value from the `Date` column.
2. **Month**: Extracted as a numeric value from the `Date` column.
3. **Season**: Derived from the `Month` column, categorized into:
    - **Winter**: December, January, February
    - **Spring**: March, April, May
    - **Summer**: June, July, August
    - **Fall**: September, October, November
    - **Unknown**: For missing or invalid months

In [ ]:
# Now create the additional date features only for valid dates
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Season"] = df["Date"].dt.month.apply(
    lambda x: "Winter"
    if pd.notna(x) and x in [12, 1, 2]
    else "Spring"
    if pd.notna(x) and x in [3, 4, 5]
    else "Summer"
    if pd.notna(x) and x in [6, 7, 8]
    else "Fall"
    if pd.notna(x) and x in [9, 10, 11]
    else "Unknown"
)

## Understanding the Numerical Data Structure
Before diving into the conversion process, we need to establish a clear taxonomy of all numerical columns in our dataset. This approach lets us apply consistent cleaning rules based on each column's characteristics and expected value ranges.

### Why This Matters
- **Accuracy**: Different column types need different treatment (e.g., percentages vs. durations)
- **Consistency**: Applying the same logic to similar columns prevents inconsistencies
- **Maintainability**: Centralized definition makes future updates easier
- **Documentation**: Serves as self-documenting code that explains column purposes


In [ ]:
# === NUMERIC COLUMNS REFERENCE LIST ===
# This list defines all the numerical features in the dataset that need special handling.
# It's used in multiple places:
# 1. Converting string values to proper numeric types
# 2. Handling outliers and invalid values
# 3. Creating correlation matrices
# 4. Filling missing values with appropriate strategies (median)
#
# Each column type has specific cleaning rules applied:
# - For all numeric columns: Convert to float, replace negative values with 0
# - For average columns: Cap extreme values at 500 minutes (8h30)
# - For percentage columns: Ensure values are within 0-100% range

numeric_columns = [
    # Journey metrics
    "Average journey time",
    # Train count metrics
    "Number of scheduled trains",
    "Number of cancelled trains",
    "Number of trains delayed at departure",
    "Number of trains delayed at arrival",
    "Number of trains delayed > 15min",
    "Number of trains delayed > 30min",
    "Number of trains delayed > 60min",
    # Delay time metrics
    "Average delay of late trains at departure",
    "Average delay of all trains at departure",
    "Average delay of late trains at arrival",
    "Average delay of all trains at arrival",
    "Average delay of trains > 15min (if competing with flights)",
    # Delay cause percentages
    "Pct delay due to external causes",
    "Pct delay due to infrastructure",
    "Pct delay due to traffic management",
    "Pct delay due to rolling stock",
    "Pct delay due to station management and equipment reuse",
    "Pct delay due to passenger handling (crowding, disabled persons, connections)",
]

## Cleaning Each Numeric Column
To ensure the quality and consistency of numeric data, we apply the following cleaning steps to each numeric column:

1. **Convert to Numeric**: Coerce non-numeric values to `NaN` to handle invalid entries.
2. **Handle Negative Values**: Replace negative values with `0` as they are not valid in this context.
3. **Cap Extreme Values**:
   - For columns representing averages (e.g., delays), cap values at a reasonable maximum of `500` minutes (8 hours and 30 minutes).
4. **Validate Percentages**:
   - For percentage columns, ensure values are within the range of `0%` to `100%`.
   - Cap values exceeding `100%` and floor negative values to `0%`.

These steps ensure that the numeric columns are clean, consistent, and ready for analysis or modeling.

In [ ]:
# Clean up each numeric column
for col in numeric_columns:
    if col in df.columns:
        # First convert to numeric, coercing errors to NaN
        df[col] = pd.to_numeric(df[col], errors="coerce")

        # Check for negative values and set them to 0
        negative = df[col] < 0
        if negative.any():
            print(f"Found {negative.sum()} negative values in {col}")
            df.loc[negative, col] = 0.0

        # Check for obviously invalid values (like extremely large numbers)
        if "Average" in col:
            # For average columns, cap to reasonable maximum 500 minutes (8h30 max)
            max_reasonable_delay = 500.0
            outliers = df[col] > max_reasonable_delay
            if outliers.any():
                print(
                    f"Found {outliers.sum()} outliers in {col} (values > {max_reasonable_delay})"
                )
                df.loc[outliers, col] = np.nan  # Set outliers to NaN

        # For percentage columns, ensure they are within 0-100 range
        if "Pct" in col:
            # Check for values greater than 100
            over_hundred = df[col] > 100
            if over_hundred.any():
                print(f"Found {over_hundred.sum()} values over 100% in {col}")
                df.loc[over_hundred, col] = 100.0  # Cap at 100%

            # Check for negative values
            negative = df[col] < 0
            if negative.any():
                print(f"Found {negative.sum()} negative values in {col}")
                df.loc[negative, col] = 0.0  # Floor at 0%

## Handling Missing Values
In this section, we address missing values in the dataset to ensure data quality and consistency. The following steps are taken:

1. **Identify Missing Values**:
    - Columns with missing values are identified, and their percentages are calculated.
    - Columns with a high percentage of missing values (e.g., `Cancellation comments` and `Departure delay comments`) are excluded from further analysis.

2. **Imputation Strategies**:
    - **Numeric Columns**: Missing values are filled with the median of the respective column to minimize the impact of outliers.
    - **Categorical Columns**: Missing values are replaced with the most frequent value (mode) in the column.

3. **Validation**:
    - After imputation, the dataset is checked to ensure no missing values remain in the columns used for analysis.

In [ ]:
# Check for columns with very few non-null values
sparsity_threshold = (
    0.05  # Consider a column sparse if less than 5% of values are non-null
)

# Check for sparse columns
for col in df.columns:
    non_null_ratio = df[col].notna().mean()
    if non_null_ratio < sparsity_threshold:
        print(f"Column '{col}' has only {non_null_ratio:.2%} non-null values")

# Print summary of numeric columns after cleaning
print("\nSummary statistics after cleaning numeric columns:")
print(df[numeric_columns].describe().T[["count", "mean", "min", "max"]])

## Feature Engineering

In this section, we create additional features from the existing data to enhance our analysis and predictive modeling capabilities. These engineered features help us extract more value from the raw data and capture important patterns in train delays.

### Delay Categorization and Severity Metrics

We first categorize delays into meaningful groups based on their severity, which helps simplify analysis and visualization while making the patterns more interpretable:

1. **Delay Categories**: 
   - **Minimal**: Delays under 5 minutes that have negligible impact
   - **Moderate**: Delays between 5-15 minutes (noticeable but tolerable)
   - **Significant**: Delays between 15-30 minutes (disruptive to travelers)
   - **Severe**: Delays over 30 minutes (major disruption to travel plans)

2. **Delay Score Aggregation**:
   - We calculate a total delay score by combining all percentage contributions from different delay causes
   - This provides a single metric that quantifies the overall impact of various delay factors

These engineered features transform continuous delay values into more actionable insights that can drive operational improvements and better passenger communication.

In [ ]:
# Create delay categories
df["Delay_Category"] = pd.cut(
    df["Average delay of all trains at arrival"],
    bins=[-float("inf"), 5, 15, 30, float("inf")],
    labels=["Minimal", "Moderate", "Significant", "Severe"],
)

# Calculate a delay score (weighted average of different delay percentages)
delay_cause_columns = [col for col in df.columns if "Pct delay due to" in col]
df["Total_Delay_Score"] = df[delay_cause_columns].sum(axis=1, skipna=True)

## Visualization 1: Delay Distribution
Let's visualize the distribution of delays to understand their pattern.

In [ ]:
# 1. Delay distribution
plt.figure(figsize=(10, 6))

# Calculate bin edges at 5-minute intervals
bin_edges = np.arange(0, 80 + 5, 5)

# Plot histogram with custom bins
sns.histplot(df["Average delay of all trains at arrival"].dropna(), bins=50, kde=True)

plt.title("Distribution of Average Delays at Arrival")
plt.xlabel("Delay (minutes)")
plt.ylabel("Frequency")
plt.xticks(bin_edges)  # Show every other tick to avoid overcrowding

# Display in notebook
plt.show()

# Save to file (optional)
plt.savefig("delay_distribution.png")
plt.close()

## Visualization 2: Monthly Trends
Let's analyze how delays have evolved over time.

In [ ]:
# 2. Monthly Trends visualization - IMPROVED X-AXIS READABILITY
plt.figure(figsize=(12, 6))
monthly_delays = (
    df.groupby(["Year", "Month"])["Average delay of all trains at arrival"]
    .mean()
    .reset_index()
)
monthly_delays["YearMonth"] = (
    monthly_delays["Year"]
    .astype(int)
    .astype(str)  # Convert to int first to remove decimals
    + "-"
    + monthly_delays["Month"]
    .astype(int)
    .astype(str)
    .str.zfill(2)  # Convert to int then to string
)
plt.plot(
    monthly_delays["YearMonth"],
    monthly_delays["Average delay of all trains at arrival"],
    marker="o",
)
plt.title("Monthly Average Delays")
plt.xlabel("Year-Month")
plt.ylabel("Average Delay (minutes)")

# Improve x-axis readability by showing only every 4th label
x_ticks = plt.xticks()[0]
x_labels = plt.xticks()[1]
step = 4  # Show every 4th label

# Keep only every Nth tick and label
plt.xticks(
    x_ticks[::step], [label.get_text() for label in x_labels][::step], rotation=45
)  # Reduced rotation from 90 to 45 degrees for better readability

plt.tight_layout()

# Display in notebook
plt.show()

# Save to file
plt.savefig("monthly_trends.png")
plt.close()

## Visualization 3: Stations with Most Delays
Let's identify which departure stations experience the most delays.

In [ ]:
# 3. Top departure stations visualization
plt.figure(figsize=(12, 8))
station_delays = (
    df.groupby("Departure station")["Average delay of all trains at departure"]
    .mean()
    .sort_values(ascending=False)
    .head(15)
)
sns.barplot(x=station_delays.values, y=station_delays.index)
plt.title("Top 15 Departure Stations with Highest Average Delays")
plt.xlabel("Average Delay (minutes)")
plt.tight_layout()

# Display in notebook
plt.show()

# Save to file
plt.savefig("station_delays.png")
plt.close()

## Visualization 4: Causes of Delays
Let's understand the main causes behind train delays.

In [ ]:
# 4. Delay causes breakdown
plt.figure(figsize=(10, 8))
delay_causes = df[delay_cause_columns].mean().sort_values(ascending=False)
sns.barplot(
    x=delay_causes.values,
    y=[col.replace("Pct delay due to ", "") for col in delay_causes.index],
)
plt.title("Average Percentage of Delays by Cause")
plt.xlabel("Average Percentage")
plt.tight_layout()

# Display in notebook
plt.show()

# Save to file
plt.savefig("delay_causes.png")
plt.close()

## Visualization 5: Correlation Analysis
Let's explore the correlations between different numeric variables.

In [ ]:
# 5. Correlation heatmap
plt.figure(figsize=(16, 14))
correlation_cols = numeric_columns
correlation = df[correlation_cols].corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))
sns.heatmap(
    correlation, mask=mask, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5
)
plt.title("Correlation Between Numerical Features")
plt.tight_layout()

# Display in notebook
plt.show()

# Save to file
plt.savefig("correlation_heatmap.png")
plt.close()

## Visualization 6: Seasonal Patterns
Let's check if there are seasonal patterns in train delays.

In [ ]:
# 6. Seasonal patterns
plt.figure(figsize=(10, 6))
seasonal_delays = (
    df.groupby("Season")["Average delay of all trains at arrival"]
    .mean()
    .reindex(["Winter", "Spring", "Summer", "Fall"])
)
sns.barplot(x=seasonal_delays.index, y=seasonal_delays.values)
plt.title("Average Delays by Season")
plt.ylabel("Average Delay (minutes)")
plt.tight_layout()

# Display in notebook
plt.show()

# Save to file
plt.savefig("seasonal_patterns.png")
plt.close()

## Final Data Preparation
Let's prepare the final dataset for modeling by handling any remaining missing values.

In [ ]:
# Final data preparation for modeling
# Create a more comprehensive feature set
print("\nPreparing final dataset for modeling...")

# Fill missing values with appropriate strategies
# For numeric columns, fill with median
for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# For categorical columns, fill with most common value
categorical_columns = ["Service", "Departure station", "Arrival station", "Season"]
for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])

# Create a binary column for significant delays (over 15 minutes)
df["Is_Significantly_Delayed"] = (
    df["Average delay of all trains at arrival"] > 15
).astype(int)

# Create a route column
df["Route"] = df["Departure station"] + " to " + df["Arrival station"]

## Save the Cleaned Dataset
Finally, let's save our cleaned dataset and summarize key insights.

In [ ]:
# Save the cleaned and processed dataset
df.to_csv("cleaned_dataset.csv", index=False)
print("\nCleaned dataset saved to 'cleaned_dataset.csv'")

# Print key insights
print("\nKey Insights from EDA:")
print(
    f"1. Average delay across all trains: {df['Average delay of all trains at arrival'].mean():.2f} minutes"
)
print(
    f"2. Percentage of trains with significant delays (>15min): {(df['Is_Significantly_Delayed'].mean() * 100):.2f}%"
)
print(
    f"3. Main cause of delays: {delay_causes.index[0].replace('Pct delay due to ', '')}"
)
print(
    f"4. Top delayed route: {df.groupby('Route')['Average delay of all trains at arrival'].mean().sort_values(ascending=False).index[0]}"
)
print(f"5. Season with most delays: {seasonal_delays.idxmax()}")
print("\nEDA complete! The dataset is now ready for predictive modeling.")